# Наумкин Владимир, С01-119.

## Задача 1. Распознавание именованных сущностей на основе fasttext.

### Подключим библиотеки

In [1]:
import fasttext # pip install fasttext-wheel
import fasttext.util
from tqdm.notebook import tqdm
import numpy as np
import torch
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter
from nerus import load_nerus
from sklearn.model_selection import train_test_split

### Уберём предупреждения

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Зададим устройство исполнения кода (вычисления провожу на своём ПК)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Код для обучения модели

In [4]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_fn):
    model.train()
    model.zero_grad()
    preds = model(x_batch.to(model.device))
    loss = loss_fn(preds.transpose(1, 2).to(model.device), y_batch.to(model.device))
    loss.backward()
    optimizer.step()
    return loss.detach().cpu().numpy()

In [5]:
def train_epoch(data_loader, model, loss_fn, optimizer, callback=None):
    total_loss = 0
    count = 0
    for i, (x, y) in enumerate(data_loader):
        batch_loss = train_on_batch(model, x, y, optimizer, loss_fn)
        data_loader.set_postfix({'current training loss': batch_loss})
        if callback != None:
            callback(model, batch_loss)
        total_loss += batch_loss * len(x)
        count += len(x)
    return total_loss / count

In [6]:
def create_batches(data, tag2index, token2index, batch_size=64, do_shuffle=True):
    tokens, tags = data
    PAD_TOK = token2index['[PAD]']
    PAD_TAG = tag2index['[PAD]']
    sample_size = len(tokens)
    indices = np.arange(sample_size)
    if do_shuffle:
        np.random.shuffle(indices)
    shuffled_tokens = [tokens[idx] for idx in indices]
    shuffled_tags = [tags[idx] for idx in indices]
    num_batches = (sample_size // batch_size) + (1 if sample_size % batch_size else 0)
    for batch_index in range(num_batches):
        end_index = min((batch_index + 1) * batch_size, sample_size)
        batch_tokens = shuffled_tokens[batch_index * batch_size:end_index]
        batch_tags = shuffled_tags[batch_index * batch_size:end_index]
        max_len = max(len(sentence) for sentence in batch_tokens)
        batch_x = np.full((end_index - batch_index * batch_size, max_len), PAD_TOK)
        batch_y = np.full((end_index - batch_index * batch_size, max_len), PAD_TAG)
        for j in range(end_index - batch_index * batch_size):
            token_ids = [token2index.get(token, token2index['O']) for token in batch_tokens[j]]
            tag_ids = [tag2index.get(tag, tag2index['O']) for tag in batch_tags[j]]
            batch_x[j, :len(token_ids)] = token_ids
            batch_y[j, :len(tag_ids)] = tag_ids
        yield torch.LongTensor(batch_x), torch.LongTensor(batch_y)

In [7]:
def trainer(num_epochs, batch_size, model, data, tag_to_index, token_to_index, loss_fn, optimizer, callback):
    epochs = tqdm(range(num_epochs))
    for epoch in epochs:
        current_optimizer = optimizer
        batch_count = len(data[0]) // batch_size + (len(data[0]) % batch_size > 0)
        batch_gen = tqdm(create_batches(data, tag_to_index, token_to_index, batch_size=batch_size, do_shuffle=True),
                         leave=False, total=batch_count)
        epoch_loss = train_epoch(data_loader=batch_gen,
                                            model=model,
                                            loss_fn=loss_fn,
                                            optimizer=current_optimizer,
                                            callback=callback)
        epochs.set_postfix({'average loss for epoch': epoch_loss})

In [8]:
class callback():
    def __init__(self, writer, dataset, tag2idx, token2idx, loss_function, delimeter=100, batch_size=64):
        self.step = 0
        self.writer = writer
        self.delimeter = delimeter
        self.loss_function = loss_function
        self.batch_size = batch_size
        self.tag2idx = tag2idx
        self.token2idx = token2idx
        self.dataset = dataset
    def forward(self, model, loss):
        self.step += 1
        self.writer.add_scalar('LOSS/train', loss, self.step)
        if self.step % self.delimeter == 1:
            real, pred = [], []
            number_of_batch = len(self.dataset[0]) // self.batch_size + (len(self.dataset[0])%self.batch_size > 0)
            generator = create_batches(self.dataset, self.tag2idx, self.token2idx, batch_size=self.batch_size)
            model.eval()
            test_loss = 0
            for it, (batch_of_x, batch_of_y) in enumerate(generator):
                batch_of_x = batch_of_x.to(model.device)
                batch_of_y = batch_of_y.to(model.device)
                with torch.no_grad():
                    output = model(batch_of_x.to(model.device))
                    test_loss += self.loss_function(output.transpose(1,2), batch_of_y).cpu().item()*len(batch_of_x)
                pred.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
                real.extend(batch_of_y.cpu().numpy().tolist())
            test_loss /= len(self.dataset[0])
            self.writer.add_scalar('LOSS/test', test_loss, self.step)
            pred4report = []
            real4report = []
            for (sent_real, sent_pred) in zip(real, pred):
                realWOpad = []
                predWOpad = []
                for (i, idx) in enumerate(sent_real):
                    if idx != self.tag2idx['[PAD]']:
                        realWOpad.append(index_to_tag[idx])
                        predWOpad.append(index_to_tag[sent_pred[i]])
                real4report.append(realWOpad)
                pred4report.append(predWOpad)
            flat_real = [item for sublist in real4report for item in sublist]
            flat_pred = [item for sublist in pred4report for item in sublist]
            self.writer.add_text('REPORT/test', str(classification_report(flat_real, flat_pred)), self.step)
            nice_sentence = "По словам Владимира Наумкина, самым лучшим учебным заведением России является МФТИ."
            sample = sample_model(model)
            self.writer.add_text('TEXT/test', nice_sentence + '\n' + sample, self.step)
    def __call__(self, model, loss):
        return self.forward(model, loss)

### Датасет NERUS

In [9]:
docs = load_nerus('nerus_lenta.conllu.gz')
tokens = []
tags = []

n_docs = 7777
for _ in range(n_docs):
    doc = next(docs)
    for sent in doc.sents:
        sent_tokens = []
        sent_tags = []
        for word in sent.tokens:
            sent_tokens.append(word.text)
            sent_tags.append(word.tag)
        tokens.append(sent_tokens)
        tags.append(sent_tags)

print(f"Number of sentences: {len(tokens)}")

Number of sentences: 92481


In [10]:
num_sentences = len(tokens)
training_tokens, testing_tokens, training_tags, testing_tags = train_test_split(tokens, tags, test_size=0.2, random_state=7)
train_data = [training_tokens, training_tags]
test_data = [testing_tokens, testing_tags]
print(f"Training set size = {len(training_tokens)}; Test set size = {len(testing_tokens)};")
print(f"Total sentences = {len(training_tokens) + len(testing_tokens)} = {num_sentences}")

Training set size = 73984; Test set size = 18497;
Total sentences = 92481 = 92481


In [11]:
tags = train_data[1]
tags_set = set(['[PAD]'])
for sent_tags in tags:
    for tag in sent_tags:
        tags_set.add(tag)
tags_list = sorted(tags_set)
print(f"{len(tags_list)} tags: {tags_list}")

8 tags: ['B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O', '[PAD]']


In [12]:
flat_tags = [item for sublist in train_data[1] for item in sublist]
tags_freq = dict((x, flat_tags.count(x) * 100 / len(flat_tags)) for x in set(flat_tags) if flat_tags.count(x) > 0)
tags_freq

{'I-PER': 1.3253905935110573,
 'I-ORG': 1.7462134896204982,
 'O': 90.15223667593203,
 'B-LOC': 2.3242291880013974,
 'B-ORG': 2.156810643132033,
 'I-LOC': 0.3637975870233087,
 'B-PER': 1.9313218227796702}

Как мы видим, большинство слов не являются именованными сущностями (тег 'O' = other). Также есть 3 типа именованных сущностей: персоны, организации и локации; каждая из которых подразделяется на начало (beginning) и внутреннюю часть (inside).

### Модель fasttext и словари

In [13]:
"""
# Не хватило оперативки на компе
# https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz
ft = fasttext.load_model('cc.ru.300.bin')
print(ft.get_dimension())
fasttext.util.reduce_model(ft, 100)
print(ft.get_dimension())
"""
# https://huggingface.co/kernela/fasttext-ru-vectors-dim-100/blob/main/ru-vectors-dim-100.bin
ft = fasttext.load_model('ru-vectors-dim-100.bin')
print(ft.get_dimension())

100


In [14]:
token_to_index = {}
index_to_token = {}
embedding_matrix = []

for index, word in enumerate(tqdm(ft.get_words(on_unicode_error='replace'))):
    vector = ft.get_word_vector(word)
    if word not in token_to_index:
        token_to_index[word] = index
        index_to_token[index] = word
        embedding_matrix.append(vector)

for special_token in ['[PAD]', '[UNK]']:
    current_index = len(token_to_index)
    token_to_index[special_token] = current_index
    index_to_token[current_index] = special_token
    embedding_matrix.append(np.zeros_like(embedding_matrix[0]))

embedding_matrix = np.array(embedding_matrix)

  0%|          | 0/2000000 [00:00<?, ?it/s]

In [15]:
tag_to_index = {tag: index for index, tag in enumerate(tags_list)}
index_to_tag = {index: tag for index, tag in enumerate(tags_list)}

### Модель

In [16]:
class RecurrentNeuralNetwork(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device
    def __init__(self,
                 vocabulary_size,
                 number_of_classes,
                 embedding_dimension=20,
                 hidden_layer_size=20,
                 layer_count=3,
                 dropout_rate=0,
                 use_batch_norm=False,
                 is_bidirectional=False):
        super(RecurrentNeuralNetwork, self).__init__()
        self.num_directions = 2 if is_bidirectional else 1
        self.embedding_dimension = embedding_dimension
        self.hidden_layer_size = hidden_layer_size
        self.use_batch_norm = use_batch_norm
        self.token_embeddings = torch.nn.Embedding(vocabulary_size, embedding_dimension)
        self.lstm = torch.nn.LSTM(embedding_dimension,
                                  hidden_layer_size,
                                  layer_count,
                                  dropout=dropout_rate,
                                  batch_first=True,
                                  bidirectional=is_bidirectional)
        self.output_layer = torch.nn.Linear(hidden_layer_size * self.num_directions, number_of_classes)
        self.batch_norm = torch.nn.BatchNorm1d(hidden_layer_size * self.num_directions) if use_batch_norm else None
    def forward(self, inputs):
        embedded_tokens = self.token_embeddings(inputs)
        lstm_output, (hidden_state, cell_state) = self.lstm(embedded_tokens)
        if self.use_batch_norm:
            lstm_output = self.batch_norm(lstm_output.transpose(1, 2)).transpose(1, 2)
        return self.output_layer(lstm_output)

Создаём модель с предобученным fasttext

In [17]:
model_config = {
    'vocabulary_size': len(token_to_index),
    'number_of_classes': len(tags_list),
    'embedding_dimension': 100,
    'hidden_layer_size': 100,
    'layer_count': 1,
    'use_batch_norm': True,
    'is_bidirectional': False
}

neural_network = RecurrentNeuralNetwork(**model_config)
neural_network.token_embeddings.weight.data.copy_(torch.tensor(embedding_matrix))
for param in neural_network.token_embeddings.parameters():
    param.requires_grad = False
neural_network = neural_network.to(device)

### Функции проверки качества модели

In [18]:
def check_model(neural_network, test_data):
    test_data_batches = create_batches(test_data, tag_to_index, token_to_index, batch_size=64)
    predicted_tags = []
    actual_tags = []
    neural_network.eval() 
    for iteration, (x_batch, y_batch) in enumerate(test_data_batches):
        x_batch = x_batch.to(device)
        with torch.no_grad():
            model_output = neural_network(x_batch)
        predicted_tags.extend(torch.argmax(model_output, dim=-1).cpu().numpy().tolist())
        actual_tags.extend(y_batch.cpu().numpy().tolist())
    processed_predictions = []
    processed_actuals = []
    for real_tags_sequence, predicted_tags_sequence in zip(actual_tags, predicted_tags):
        filtered_actual_tags = []
        filtered_predicted_tags = []
        for idx, tag_index in enumerate(real_tags_sequence):
            if tag_index != tag_to_index['[PAD]']:  # Не учитываем паддинг
                filtered_actual_tags.append(index_to_tag[tag_index])
                filtered_predicted_tags.append(index_to_tag[predicted_tags_sequence[idx]])
        processed_actuals.append(filtered_actual_tags)
        processed_predictions.append(filtered_predicted_tags)
    flat_actual_tags = [tag for sublist in processed_actuals for tag in sublist]
    flat_predicted_tags = [tag for sublist in processed_predictions for tag in sublist]
    print(classification_report(flat_actual_tags, flat_predicted_tags))

In [34]:
def sample_model(model, sentence = ['По', 'словам', 'Владимира', 'Наумкина', ',',
                                    'самым', 'лучшим', 'учебным', 'заведением', 'России',
                                    'является', 'МФТИ', '.']):
    # Преобразуем токены в индексы
    token_indices = [token_to_index.get(token, token_to_index['[UNK]']) for token in sentence]
    #token_indices = [token_to_index[token] for token in sentence] # Изначально было так (во время обучения),
                                                                   # хорошо, что не нарвался на ошибку
                                                                   # заметил только при финальном тесте (см. выводы)
    # Преобразуем список индексов в тензор
    input_tensor = torch.tensor(token_indices, dtype=torch.long).unsqueeze(0).to(model.device)  # Добавляем размерность для батча
    # Получаем предсказания от модели
    model.eval()
    with torch.no_grad():  # Не нужно вычислять градиенты
        output = model(input_tensor)
    # Получаем индексы с максимальным значением для каждого токена
    predicted_indices = output.argmax(dim=2).squeeze().cpu().numpy()  # Убираем размерность батча
    # Преобразуем индексы в теги
    predicted_tags = [index_to_tag[index] for index in predicted_indices]
    # Объединяем теги в одну строку
    tags_string = ' '.join(predicted_tags)
    return tags_string

## Обучение

In [20]:
loss_function = torch.nn.CrossEntropyLoss(ignore_index=tag_to_index['[PAD]'])
optimizer = torch.optim.Adam(neural_network.parameters(), lr=1e-3)

Качество до обучения

In [21]:
check_model(neural_network, test_data)

              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00      7776
       B-ORG       0.02      1.00      0.04      7388
       B-PER       0.00      0.00      0.00      6396
       I-LOC       0.00      0.00      0.00      1254
       I-ORG       0.00      0.00      0.00      5813
       I-PER       0.00      0.00      0.00      4340
           O       0.00      0.00      0.00    301633

    accuracy                           0.02    334600
   macro avg       0.00      0.14      0.01    334600
weighted avg       0.00      0.02      0.00    334600



In [22]:
# По словам Владимира Наумкина, самым лучшим учебным заведением России является МФТИ.
# O  O      B-PER     I-PER   O O     O      O       O          B-LOC  O        B-ORG O
print(sample_model(neural_network))

B-ORG B-ORG B-ORG B-ORG B-ORG B-ORG B-ORG B-ORG B-ORG B-ORG B-ORG B-ORG B-ORG


In [23]:
writer = SummaryWriter(log_dir='tensorboard1/layer1_dim100_dropout0_bnormT')
call = callback(writer, test_data, tag_to_index, token_to_index, loss_function, delimeter = 70)

In [24]:
trainer(
    num_epochs=10,
    batch_size=64,
    data=train_data,
    model=neural_network,
    tag_to_index=tag_to_index,
    token_to_index=token_to_index,
    loss_fn=loss_function,
    optimizer=optimizer,
    callback=call
)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1156 [00:00<?, ?it/s]

  0%|          | 0/1156 [00:00<?, ?it/s]

  0%|          | 0/1156 [00:00<?, ?it/s]

  0%|          | 0/1156 [00:00<?, ?it/s]

...

  0%|          | 0/1156 [00:00<?, ?it/s]

  0%|          | 0/1156 [00:00<?, ?it/s]

  0%|          | 0/1156 [00:00<?, ?it/s]

  0%|          | 0/1156 [00:00<?, ?it/s]

  0%|          | 0/1156 [00:00<?, ?it/s]

Качество после обучения

In [25]:
check_model(neural_network, test_data)

              precision    recall  f1-score   support

       B-LOC       0.94      0.96      0.95      7776
       B-ORG       0.91      0.86      0.89      7388
       B-PER       0.97      0.92      0.94      6396
       I-LOC       0.91      0.89      0.90      1254
       I-ORG       0.87      0.88      0.87      5813
       I-PER       0.96      0.98      0.97      4340
           O       0.99      1.00      0.99    301633

    accuracy                           0.99    334600
   macro avg       0.94      0.93      0.93    334600
weighted avg       0.99      0.99      0.99    334600



In [26]:
# По словам Владимира Наумкина, самым лучшим учебным заведением России является МФТИ.
# O  O      B-PER     I-PER   O O     O      O       O          B-LOC  O        B-ORG O
print(sample_model(neural_network))

O O B-PER I-PER O O O O O B-LOC O B-ORG O


## Выводы

Модель почти сразу начала правильно определять именованные сущности в моём предложении. Возможно, оно оказалось достаточно коротким и простым. Если же смотреть на classification report и графики loss, то можно сказать, что модель постепенно улучшалась и как раз начала выходить на свой максимум качества.

Для полноты картины выведем примеры работы на тестовых предложениях:

In [32]:
for i in range(5):
    tok4check = test_data[0][i]
    tag4check = test_data[1][i]
    print(tok4check)
    print(tag4check)
    print()
    print(sample_model(neural_network, tok4check))
    print()
    print()

['«', 'Президента', '»', 'Унгер', 'и', 'еще', '26', 'человек', 'задержали', 'в', 'апреле', '2017', 'года', 'в', 'результате', 'полицейских', 'рейдов', 'по', 'всей', 'стране', '.']
['O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

O B-ORG O B-PER O O O O O O O O O O O O O O O O O


['Как', 'отмечают', 'эксперты', ',', 'опасности', 'для', 'банка', 'подобная', 'утечка', 'не', 'представляет', ',', 'но', 'сотрудники', 'могут', 'стать', 'мишенями', 'для', 'массовой', 'рассылки', 'фишинговых', 'писем', ',', 'рекламы', 'и', 'спама', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

O O O O O O O O O O O O O O O O O O O O O O O O O O O


['По', 'данным', 'ЦИК', ',', 'за', 'нее', 'проголосовали', '59,56', 'процентов', 'граждан', 'Грузии', '.']
['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O']

O O B-ORG O O O O O O O B-LOC O



In [62]:
final_sentence = 'В солнечный день в парке имени Пушкина в центре Москвы, где гуляли дети из школы номер 12, встречались друзья Анна, Дмитрий и Екатерина, обсуждая последние новости из жизни своих любимых артистов, таких как Сергей Безруков и Анастасия Заворотнюк, а также планы на выходные, которые они собирались провести в театре имени Маяковского, наслаждаясь спектаклем по пьесе Чехова, после чего собирались отправиться на экскурсию в музей современного искусства "Арт-Гармония" в Санкт-Петербурге и посетить выставку, организованную Международной ассоциацией художников "Творчество без границ".'

In [63]:
from nltk.tokenize import RegexpTokenizer

In [64]:
final_toks = RegexpTokenizer('[а-яА-Я]+|[^\w\s]|\d+').tokenize(final_sentence)

In [65]:
def final_sample_model(model, sentence):
    token_indices = [token_to_index.get(token, token_to_index['[UNK]']) for token in sentence]
    input_tensor = torch.tensor(token_indices, dtype=torch.long).unsqueeze(0).to(model.device)
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
    predicted_indices = output.argmax(dim=2).squeeze().cpu().numpy()
    predicted_tags = [index_to_tag[index] for index in predicted_indices]
    return predicted_tags

In [66]:
final_tags = final_sample_model(neural_network, final_toks)

In [67]:
for i in range(len(final_toks)):
    print(final_toks[i], final_tags[i])

В O
солнечный O
день O
в O
парке O
имени O
Пушкина O
в O
центре O
Москвы B-LOC
, O
где O
гуляли O
дети O
из O
школы O
номер O
12 O
, O
встречались O
друзья O
Анна B-PER
, O
Дмитрий B-PER
и O
Екатерина B-PER
, O
обсуждая O
последние O
новости O
из O
жизни O
своих O
любимых O
артистов O
, O
таких O
как O
Сергей B-PER
Безруков I-PER
и O
Анастасия B-PER
Заворотнюк I-PER
, O
а O
также O
планы O
на O
выходные O
, O
которые O
они O
собирались O
провести O
в O
театре O
имени O
Маяковского O
, O
наслаждаясь O
спектаклем O
по O
пьесе O
Чехова B-PER
, O
после O
чего O
собирались O
отправиться O
на O
экскурсию O
в O
музей O
современного O
искусства O
" O
Арт O
- O
Гармония B-ORG
" O
в O
Санкт B-LOC
- O
Петербурге B-LOC
и O
посетить O
выставку O
, O
организованную O
Международной B-ORG
ассоциацией I-ORG
художников I-ORG
" I-ORG
Творчество I-ORG
без O
границ O
" O
. O


Даже со сгенерированным ГПТишкой предложением модель, по моему мнению, в целом отлично справилась, хотя жанр предложения - это скорее проза, а не новость.